In [2]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]
pred_sklearn = clr.predict(X_test)
display(pred_onx)
display(pred_sklearn)
display(pred_onx==pred_sklearn)

array([0, 1, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 2, 0, 1, 2, 0, 2, 1, 1, 1, 0,
       0, 0, 0, 2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1, 2, 2], dtype=int64)

array([0, 1, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 2, 0, 1, 2, 0, 2, 1, 1, 1, 0,
       0, 0, 0, 2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1, 2, 2])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [3]:
import onnx
import warnings
from onnx_tf.backend import prepare

warnings.filterwarnings('ignore') 
onnx_model = onnx.load("rf_iris.onnx")  # load onnx model
prepared = prepare(onnx_model)
prepared

/onnx-tensorflow/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


KeyError: 'ai.onnx.ml'

In [4]:
import tensorflow
tensorflow.__version__

'2.1.0'